In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [11]:
df = pd.read_csv('Car_Prices_Poland_Kaggle.csv')

In [12]:
data = df.copy()

In [13]:
data.drop('Unnamed: 0', axis = 1, inplace = True)

In [17]:
# Label Encoding 
encoder = LabelEncoder()
data['mark'] = encoder.fit_transform(data['mark'])
data['model'] = encoder.fit_transform(data['model'])
data['generation_name'] = encoder.fit_transform(data['generation_name'])
data['province'] = encoder.fit_transform(data['province'])
data['fuel'] = encoder.fit_transform(data['fuel'])
data['city'] = encoder.fit_transform(data['city'])

In [19]:
# Removing the outliers
data = data[data['price'].between(data['price'].quantile(.15), data['price'].quantile(.85))] 

In [22]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
import math

In [24]:
# Feature Selection with boruta
rf = RandomForestRegressor( n_jobs=-1 )
boruta = BorutaPy( rf, n_estimators='auto', verbose=2, random_state=42 ).fit(data.iloc[:,0:9].values, data.iloc[:,9])

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	7
Tentative: 	0
Rejected: 	2


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	7
Tentative: 	0
Rejected: 	2


In [27]:
# columns selected by boruta
columns_selected = boruta.support_.tolist()

In [26]:
x = data.drop('price', axis = 1)

In [30]:
cols_selected_boruta = x.iloc[:, columns_selected].columns.to_list()

In [31]:
# Columns selected by boruta
cols_selected_boruta

['mark', 'model', 'generation_name', 'year', 'mileage', 'vol_engine', 'city']

In [35]:
# Not selected by boruta
cols_not_selected_boruta = list( np.setdiff1d( x.columns, cols_selected_boruta) )
cols_not_selected_boruta

['fuel', 'province']

In [36]:
x = data.drop(['province', 'fuel'], axis =1)

In [41]:
# defining the variables independent and target
x_to_train = x.iloc[:,:-1].values
y = x['price'].values

In [44]:
from sklearn.linear_model import LinearRegression

In [48]:
# Dividing data in train and test
x_train, x_test, y_train, y_test = train_test_split(x_to_train, y, test_size = 0.2, random_state = 42)

In [49]:
# Linear model regression 
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [53]:
# score in train data
model.score(x_train, y_train)

0.6769630255967438

In [54]:
# score in test data
model.score(x_test, y_test)

0.6622876624591485

In [ ]:
# Linear regression score: 67% in train and 66 in test
# Underfiting : the model is so simple, let's try a model more complex than it

In [55]:
from sklearn.ensemble import RandomForestRegressor

In [56]:
# creating the Random Forest Regressor model
forest = RandomForestRegressor()
forest.fit(x_train, y_train)

RandomForestRegressor()

In [57]:
# Score in train data
forest.score(x_train, y_train)

0.9865842662047755

In [60]:
# Score in test data
forest.score(x_test, y_test)

0.9148083372031881

In [74]:
from sklearn.metrics import mean_absolute_error

In [70]:
# Predictions of both models
linear_pred = model.predict(x_test)
forest_pred = forest.predict(x_test)

In [75]:
metric_linear = mean_absolute_error(linear_pred, y_test)
metric_forest = mean_absolute_error(forest_pred, y_test)

In [76]:
# comparing the both models with Mean Absolute Error
metric_linear

11611.949570856417

In [77]:
metric_forest

5325.390451739751